# Task2. 预测酶是单功能酶还是多功能酶

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-09-28  

## 任务简介
该任务通过给定酶序列，预测该酶是几单功能酶。


## 1. 导入必要的包

In [1]:
import numpy as np
import pandas as pd


import sys
import os
from tqdm import tqdm
from functools import reduce

sys.path.append("../../tools/")
import commontools
import funclib
import ucTools

sys.path.append("../../")
import benchmark_common as bcommon
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_config as cfg
import benchmark_evaluation as eva


from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. 加载数据

In [2]:
#读取所有数据的训练集，测试集
train = pd.read_feather(cfg.DATADIR+'train.feather')
test = pd.read_feather(cfg.DATADIR+'test.feather')

#取酶数据
train = train[train.functionCounts>0]
test =test[test.functionCounts>0]
print('train size: {0}\ntest size: {1}'.format(len(train), len(test)))

trainset = train[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)

train size: 222566
test size: 3579


## 取训练测试数据

In [6]:
#读取esm embedding 结果
train_esm_32 = pd.read_feather(cfg.DATADIR + 'train_rep32.feather')
test_esm_32 = pd.read_feather(cfg.DATADIR + 'test_rep32.feather')

train_esm = trainset.merge(train_esm_32, on='id', how='left')
test_esm = testset.merge(test_esm_32, on='id', how='left')

#取训练测试数据
X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

## 3. 同源比对

In [8]:
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd


diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 80
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /tmp/train.fasta
Opening the database file...  [0.004s]
Loading sequences...  [0.431s]
Masking sequences...  [0.197s]
Writing sequences...  [0.097s]
Hashing sequences...  [0.027s]
Loading sequences...  [0s]
Writing trailer...  [0.001s]
Closing the input file...  [0.002s]
Closing the database file...  [0.023s]
Database hash = 3c4bde718063d5a6f31d5ecc0caa2d24
Processed 222566 sequences, 93540547 letters.
Total time = 0.787s
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 80
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /tmp
#Target sequences to report alignments for:

diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


 [0.033s]
Algorithm: Double-indexed
Building query histograms...  [0.013s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.177s]
Masking reference...  [0.229s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.159s]
Allocating buffers...  [0s]
Processing query block 1, reference block 1/1, shape 1/2.
Building reference seed array...  [0.139s]
Building query seed array...  [0.019s]
Computing hash join...  [0.051s]
Building seed filter...  [0.005s]
Searching alignments...  [0.03s]
Processing query block 1, reference block 1/1, shape 2/2.
Building reference seed array...  [0.091s]
Building query seed array...  [0.014s]
Computing hash join...  [0.055s]
Building seed filter...  [0.005s]
Searching alignments...  [0.027s]
Deallocating buffers...  [0.046s]
Clearing query masking...  [0s]
Computing alignments...  [0.59s]
Deallocating reference...  [0.008s]
Loading reference sequences...  [0s]
Deallocating buffers...  [0.002s]
Deallocating queries...  [0

In [9]:
res_data

,id,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Q2U0J6,A0A1B4XBH0,36.8,506,297,8,18,511,17,511,3.420000e-99,311.0
1,A0A2I6PIZ2,Q672V4,55.3,443,191,4,5,440,6,448,1.990000e-170,489.0
2,D1MX87,S0E2U7,39.7,441,247,6,86,518,11,440,1.670000e-98,306.0
3,A0A075TR33,A1CFM2,74.0,542,141,0,25,566,26,567,2.200000e-311,855.0
4,Q2U0K0,M2YJD1,39.2,477,258,11,38,497,57,518,3.080000e-106,328.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3025,Q9NTI2,Q9NTI2,100.0,1148,0,0,41,1188,1,1148,0.000000e+00,2283.0
3026,D4AA47,Q148W0,98.1,1251,24,0,1,1251,1,1251,0.000000e+00,2416.0
3027,F1RBC8,Q94FB9,32.0,604,351,11,97,687,100,656,2.230000e-93,320.0
3028,Q7JUN3,Q94FB9,32.9,617,370,10,110,721,87,664,3.670000e-83,290.0


In [10]:
#匹配查询结果
data_match = pd.merge(test,res_data, on=['id'], how='inner')

# 添加查询结果的EC号
resArray =[]
for i in tqdm(range(len(res_data))):
    mn = train[train['id']== res_data['sseqid'][i]]['ec_number'].values
    resArray.append(mn)
data_match['sresults_ec']=np.array(resArray) 
data_match.head(3)



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3030/3030 [00:37<00:00, 79.86it/s]


,id,isemzyme,functionCounts,ec_number,ec_specific_level,seq,seqlength,f1,f2,f3,...,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,sresults_ec
0,Q2U0J6,True,1,1.-.-.-,1,MASLTGAAAVLLAFILAYSTALTIYRLFFHPLARFPGPRLAAATKW...,521,0.000223,-0.094108,-0.023778,...,506,297,8,18,511,17,511,3.420000e-99,311.0,1.-.-.-
1,A0A2I6PIZ2,True,1,1.-.-.-,1,MSTPEFKVIIVGGSLAGLTLAHCLLRAGISHIVLERRSVIAPEEGA...,463,0.001232,-0.138543,0.053197,...,443,191,4,5,440,6,448,1.990000e-170,489.0,1.-.-.-
2,D1MX87,True,1,1.-.-.-,1,MLEGTLQDCWTSISKMQLHWTVLGLLPVLFIAILGPRVRQIWVNYV...,523,0.000639,0.001423,0.113487,...,441,247,6,86,518,11,440,1.670000e-98,306.0,1.-.-.-


In [11]:
# 计算指标
data_match['iscorrect'] = data_match[['ec_number', 'sresults_ec']].apply(lambda x: x['ec_number'] == x['sresults_ec'], axis=1) #判断EC号是否一致
correct = sum(data_match['iscorrect'])
find  = len(data_match)
total = len(test)
print('Total query records are: {0}'.format(total))
print('Matched records are: {0}'.format(find))
print('Accuracy: {0}({1}/{2})'.format(correct/total, correct, total))
print('Pricision: {0}({1}/{2})'.format(correct/find, correct, find))
print('Recall: {0}({1}/{2})'.format(find/total, find, total))

Total query records are: 3579
Matched records are: 3030
Accuracy: 0.46465493154512433(1663/3579)
Pricision: 0.5488448844884488(1663/3030)
Recall: 0.8466051969823973(3030/3579)


## 4. 机器学习方法预测
### 4.1 onehot + 机器学习

In [7]:
trainset = train[['id', 'isMultiFunctional','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isMultiFunctional','seq', 'seqlength']].reset_index(drop=True)

MAX_SEQ_LENGTH = 1500 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

In [8]:
f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

X_train = train_full.iloc[:,4:]
X_test = test_full.iloc[:,4:]
Y_train = train_full.isMultiFunctional.astype('int')
Y_test = test_full.isMultiFunctional.astype('int')

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)


In [9]:
funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.910640 	0.271739 		0.922299 	0.059988 	0.098280 	0.551935 	0.124858 	 tp: 100 fp: 268 fn: 1567 tn: 18600
xg 		0.918578 	0.490909 		0.924383 	0.080984 	0.139032 	0.626453 	0.193860 	 tp: 135 fp: 140 fn: 1532 tn: 18728
dt 		0.851668 	0.173378 		0.929386 	0.219556 	0.193753 	0.563515 	0.101421 	 tp: 366 fp: 1745 fn: 1301 tn: 17123
rf 		0.924032 	0.957265 		0.923842 	0.067187 	0.125561 	0.668622 	0.269895 	 tp: 112 fp: 5 fn: 1555 tn: 18863
gbdt 		0.917507 	0.398496 		0.920890 	0.031794 	0.058889 	0.601392 	0.161334 	 tp: 53 fp: 80 fn: 1614 tn: 18788


### 4.2 Unirep + 机器学习

In [10]:
X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]

Y_train = train.iloc[:,3].astype('int')
Y_test = test.iloc[:,3].astype('int')

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

In [11]:
funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.906306 	0.268468 		0.924024 	0.089382 	0.134113 	0.658508 	0.161993 	 tp: 149 fp: 406 fn: 1518 tn: 18462
xg 		0.922669 	0.613181 		0.928019 	0.128374 	0.212302 	0.730793 	0.294830 	 tp: 214 fp: 135 fn: 1453 tn: 18733
dt 		0.867835 	0.204738 		0.930498 	0.217756 	0.211047 	0.571513 	0.108084 	 tp: 363 fp: 1410 fn: 1304 tn: 17458
rf 		0.923789 	0.947368 		0.923657 	0.064787 	0.121280 	0.693704 	0.297006 	 tp: 108 fp: 6 fn: 1559 tn: 18862
gbdt 		0.918578 	0.474747 		0.920728 	0.028194 	0.053228 	0.649263 	0.171179 	 tp: 47 fp: 52 fn: 1620 tn: 18816
